In [1]:
# импортируем модуль
import os
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time


In [ ]:

# Укажите путь к chromedriver, если он не добавлен в PATH
browser = webdriver.Chrome()

# Открываем веб-страницу
browser.get('https://grls.rosminzdrav.ru/Default.aspx')

# Находим элемент по классу и кликаем на него
link = browser.find_element(By.CLASS_NAME, 'sidebar-item__link')
link.click()

# Ждем, чтобы новая страница загрузилась
time.sleep(5)

# Находим текстовое поле по ID
input_field = browser.find_element(By.ID, "ctl00_plate_txtRecordOnPageCount")
input_field.click()
time.sleep(5)

# Изменяем значение value на "99"
input_field.clear()  # Очищаем текущее значение
input_field.send_keys("99")  # Вводим новое значение
time.sleep(5)

# Если необходимо, можно имитировать нажатие клавиши "Enter"
input_field.send_keys(Keys.RETURN)

# Находим текстовое поле по ID и кликаем на него
input_field = browser.find_element(By.ID, "ctl00_plate_txtMNN")

input_field.click()

# Вводим текст в текстовое поле
input_field.send_keys("Вакцина")  

# Если необходимо, можно имитировать нажатие клавиши "Enter"
input_field.send_keys(Keys.RETURN)

# Закрываем браузер через некоторое время
time.sleep(10)

In [ ]:
# Находим все строки таблицы
rows = browser.find_elements(By.CSS_SELECTOR, 'tr.hi_sys.poi')

# Проходим по каждой строке и извлекаем данные
row_data = []
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    data = [col.text for col in columns]
    row_data.append(data)


    

In [54]:
row_onclick = []
for row in rows:
    # Извлекаем значение onclick
    onclick_value = row.get_attribute('onclick')
    match = re.search(r"det\('(.+?)',", onclick_value)
    if match:
        extracted_id = match.group(1)  # Извлекаем первую группу
        row_onclick.append(extracted_id)  # Добавляем ID в данные строки

In [63]:
df = pd.DataFrame(row_data, columns=['№ п/п',	
                                     'Торговое наименование',
                                     'Международное непатентованное наименование или группировочное (химическое) наименование',	
                                     'Форма выпуска',	
                                     'Наименование держателя или владельца регистрационного удостоверения лекарственного препарата',	
                                     'Страна держателя или владельца регистрационного удостоверения лекарственного препарата',	
                                     'Регистрационный номер',	
                                     'Дата государственной регистрации',	
                                     'Дата окончания действ. рег. уд.',	
                                     'Дата переоформления',
                                     'Состояние',	
                                     'Дата решения'])
df['ID'] = row_onclick
df

,№ п/п,Торговое наименование,Международное непатентованное наименование или группировочное (химическое) наименование,Форма выпуска,Наименование держателя или владельца регистрационного удостоверения лекарственного препарата,Страна держателя или владельца регистрационного удостоверения лекарственного препарата,Регистрационный номер,Дата государственной регистрации,Дата окончания действ. рег. уд.,Дата переоформления,Состояние,Дата решения,ID
0,1,ИНФЛЮВИР Вакцина гриппозная живая моновалентная,Вакцина для профилактики гриппа [живая],лиофилизат для приготовления раствора для интр...,"Акционерное общество ""Научно-производственное ...",Россия,ЛП-№(008575)-(РГ-RU),24.01.2025,,,Д,24.01.2025,0ba24732-9dc8-464f-9334-1656fb2a15d4
1,2,Гам-VLP-мультивак Вакцина на основе вирусоподо...,Вакцина для профилактики COVID-19,эмульсия для внутримышечного введения;,"ФГБУ ""НИЦЭМ им. Н.Ф. Гамалеи"" Минздрава России...",Россия,ЛП-008863,06.12.2024,01.01.2028,,Д,06.12.2024,f035b599-e2f5-41c1-9ede-a7398483a6c4
2,3,КОКАВ Вакцина антирабическая культуральная кон...,Вакцина для профилактики бешенства,лиофилизат для приготовления раствора для внут...,"Акционерное общество ""Научно-производственное ...",Россия,ЛП-№(007968)-(РГ-RU),05.12.2024,,,Д,05.12.2024,7da83465-3e51-4a42-99a7-7aff7e64f9e4
3,4,Рота-V-Эйд®,Вакцина для профилактики ротавирусной инфекции...,лиофилизат для приготовления раствора для прие...,"Общество с ограниченной ответственностью ""ФАРМ...",Россия,ЛП-№(007440)-(РГ-RU),29.10.2024,29.10.2029,,Д,29.10.2024,3e6a77c5-d2ae-4f34-b737-7803e65abe65
4,5,СОВИГРИПП® Вакцина гриппозная инактивированная...,Вакцина для профилактики гриппа [инактивирован...,раствор для внутримышечного введения;,"Акционерное общество ""Научно-производственное ...",Россия,ЛП-№(007255)-(РГ-RU),15.10.2024,,,Д,15.10.2024,b287b682-550a-418f-8189-05dac16da4ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,ПАНДЕФЛЮ Вакцина гриппозная инактивированная с...,Вакцина для профилактики гриппа [инактивирован...,суспензия для внутримышечного введения;,"Акционерное общество ""Научно-производственное ...",Россия,ЛСР-007987/09,13.10.2009,,17.07.2020,Д,17.07.2020,f4c374e9-0954-4d79-85c4-148a3b816718
95,96,Вакцина желтой лихорадки живая сухая,Вакцина для профилактики желтой лихорадки,лиофилизат для приготовления раствора для подк...,Федеральное государственное автономное научное...,Россия,ЛС-000592,28.09.2009,,16.02.2022,Д,28.12.2023,a0eece39-1af9-4420-be24-7ff034677feb
96,97,Вакцина против кори живая аттенуированная,Вакцина для профилактики кори,лиофилизат для приготовления раствора для подк...,Серум Инститьют оф Индия Пвт. Лтд,Индия,ЛСР-005239/09,30.06.2009,,14.04.2023,Д,07.02.2024,0fb5a06d-2df7-4647-98fa-055df85c6963
97,98,Вакцина клещевого энцефалита культуральная очи...,Вакцина для профилактики клещевого энцефалита,лиофилизат для приготовления суспензии для вну...,Федеральное государственное автономное научное...,Россия,Р N003793/01,26.05.2009,,19.01.2023,Д,19.01.2023,48aa0e31-2ad9-4072-8261-154769e437ef


In [15]:
browser.quit()